# Convert TensroFlow saved_model to [TensorRT](https://github.com/NVIDIA/TensorRT/tree/main/samples/python/tensorflow_object_detection_api) Engine

define model location

In [1]:
workdir = '/home/edurso/dev/obj-detect/models/trained_model/'
mname = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'

download `trained_model` from google drive to `workdir`

## Installation & Environment Configuration

install tfod api

In [2]:
import os
import pathlib

os.chdir('..')

if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3344, done.
remote: Counting objects: 100% (3344/3344), done.
remote: Compressing objects: 100% (2780/2780), done.
remote: Total 3344 (delta 882), reused 1390 (delta 513), pack-reused 0
Receiving objects: 100% (3344/3344), 34.34 MiB | 7.24 MiB/s, done.
Resolving deltas: 100% (882/882), done.


In [3]:
%%bash

# update pip
pip3 install --upgrade pip

# check opencv dependency versions
# pip3 uninstall opencv-python-headless==4.5.5.62 
# pip3 install opencv-python-headless==4.1.2.30
# pip3 install onnx-graphsurgeon==0.3.10 --index-url https://pypi.ngc.nvidia.com

# compile protos and install object_detection
cd $(pwd)/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python3 -m pip install --use-feature=2020-resolver .

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Processing /home/lightning/dev/obj-detect/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached Cython-0.29.29-cp36-cp36m-manylinux_2_17_aarch64.manylinux2014_aarch64.manylinux_2_24_aarch64.whl (1.8 MB)
  Using cached apache_beam-2.38.0-cp36-cp36m-manylinux2014_aarch64.whl (11.2 MB)
  Using cached avro-python3-1.10.2.tar.gz (38 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached avro-python3-1.10.1.tar.gz (38 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached avro-python3-1.10.0.tar.gz (37 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached avro-pytho

  ERROR: Command errored out with exit status 1:
   command: /usr/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-k_ufu85e/avro-python3_cfea9685185640f5bac2cfb6e2fd332e/setup.py'"'"'; __file__='"'"'/tmp/pip-install-k_ufu85e/avro-python3_cfea9685185640f5bac2cfb6e2fd332e/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-a03e4dv8
       cwd: /tmp/pip-install-k_ufu85e/avro-python3_cfea9685185640f5bac2cfb6e2fd332e/
  Complete output (5 lines):
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "/tmp/pip-install-k_ufu85e/avro-python3_cfea9685185640f5bac2cfb6e2fd332e/setup.py", line 41, in <module>
      import pycodestyle
  ModuleNotFoundError: No mod

CalledProcessError: Command 'b'\n# update pip\npip3 install --upgrade pip\n\n# check opencv dependency versions\n# pip3 uninstall opencv-python-headless==4.5.5.62 \n# pip3 install opencv-python-headless==4.1.2.30\n# pip3 install onnx-graphsurgeon==0.3.10 --index-url https://pypi.ngc.nvidia.com\n\n# compile protos and install object_detection\ncd $(pwd)/models/research/\nprotoc object_detection/protos/*.proto --python_out=.\ncp object_detection/packages/tf2/setup.py .\npython3 -m pip install --use-feature=2020-resolver .\n'' returned non-zero exit status 1.

In [ ]:
!pip show object-detection

get tensorrt utility files

In [ ]:
url = 'https://raw.githubusercontent.com/NVIDIA/TensorRT/main/samples/python/tensorflow_object_detection_api/{}.py'
files = ['build_engine', 'create_onnx', 'infer', 'compare_tf', 'infer', 'onnx_utils', 'visualize']
os.chdir('../scripts')
for file in files:
    furl = url.format(file)
    !wget {furl}

## Create TensorRT Engine

In [ ]:
os.chdir('../scripts')
!python3 create_onnx.py \
    --pipeline_config {workdir}pipeline.config \
    --saved_model {workdir}saved_model \
    --onnx {workdir}model.onnx

In [ ]:
os.chdir('../scripts')
!python3 build_engine.py \
    --onnx {workdir}model.onnx \
    --engine {workdir}engine.trt \
    --precision fp16

## Test TensorRT Engine

In [ ]:
!trtexec \
    --loadEngine={workdir}engine.trt \
    --useCudaGraph --noDataTransfers \
    --iterations=100 --avgRuns=100

In [ ]:
os.chdir('../scripts')
!python3 infer.py \
    --engine {workdir}engine.trt \
    --input ../img \
    --output {workdir}/output \
    --preprocessor fixed_shape_resizer \
    --labels {workdir}label_map.txt

## Clean Up

In [ ]:
os.chdir('../scripts')
for file in files:
    fname = '{}.py'.format(file)
    !rm -rf {fname}